In [1]:
import numpy as np

# np.ravel()

In [2]:
x = np.array([[1,2,3], [4,5,6]])
x.ravel()

array([1, 2, 3, 4, 5, 6])

# np.stack()

In [10]:
a = np.arange(6).reshape(2,3)
b = a + 1
c = a + 2
print(a)
print('----------')
print(b)
print('----------')
a_b = np.stack((a.ravel(), b.ravel(), c.ravel()), axis=1) #按列拼接
a_b

[[0 1 2]
 [3 4 5]]
----------
[[1 2 3]
 [4 5 6]]
----------


array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6],
       [5, 6, 7]])

# np.transpose()

In [12]:
x = np.ones((1,2,3))
print(x)
x.transpose((1,0,2)).shape #1 -> 2 // 0 -> 1 // 2 -> 3

[[[1. 1. 1.]
  [1. 1. 1.]]]


(2, 1, 3)

# 广播机制

In [13]:
anchor_base = np.arange(12).reshape(1,3,4)
shift = np.arange(24).reshape(1,6,4).transpose((1,0,2))
print(anchor_base.shape, shift.shape)

(1, 3, 4) (6, 1, 4)


In [14]:
anchor = anchor_base + shift
anchor.shape

(6, 3, 4)

# model/Region_proposal_network.py

In [ ]:
def generate_anchor_base(base_size=16,
                         ratios=[0.5, 1, 2],
                         anchor_scales=[8,16,32]):
  """默认生成左上角9个anchor"""
  py = base_size / 2.
  px = base_size / 2.

  anchor_base = np.zeros((len(ratios) * len(anchor_scales), 4),
                         dtype=np.float32)
  for i in range(len(ratios)):
    for j in range(len(anchor_scales)):
      # 生成9种不同比例的h和w
      h = base_size * anchor_scales[i] * np.sqrt(ratios[i])
      w = base_size * anchor_scales[j] * np.sqrt(1. / ratios[i])

      index = i * len(anchor_scales) + j
      # 计算出anchor_base画的9个框的左下角和右上角的4个anchor坐标值
      anchor_base[index,0] = py - h/2
      anchor_base[index,1] = px - w/2
      anchor_base[index,2] = py + h/2
      anchor_base[index,3] = px + w/2

    return anchor_base

In [ ]:
class RegionProposalNetwork(nn.Module):
  """
    1.计算回归坐标偏移量
    2.计算anchors前景概率
    3.调用ProposalCreator和使用NMS得出2000个近似目标框的坐标
  """
  def __init__(self,
               in_channels=512,
               mid_channels=512,
               ratios=[0.5, 1, 2],
               anchor_scales=[8,16,32],
               feat_stride=16, #featuremap缩小的比例
               proposal_creator_params=dict()):
    super(RegionProposalNetWork,self).__init__()
    self.anchor_base = generate_anchor_base(
        anchor_scales=anchor_scales,ratios=ratios)
    self.feat_stride = feat_stride
    self.proposal_layer = ProposalCreator(self, **proposal_creator_params) #输出最后的2000
    n_anchor = self.anchor_base.shape[0]
    self.conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
    self.score = nn.Conv2d(in_channels, n_anchor*2, 1, 1, 0)
    self.loc = nn.Conv2d(mid_channels, n_anchor*4, 1, 1, 0)
    # 作归一化处理
    normal_init(self.conv1, 0, 0.01)
    normal_init(self.score, 0, 0.01)
    normal_init(self.loc, 0, 0.01)

  def forward(self, x, img_size, scale=1):
    n, _, hh, ww = x.shape #(batch_size, 512, H/16, W/16)
    anchor = _enumerate_shifted_anchor(
        np.array(self.anchor_base),
        self.feat_stride,hh,ww) # 在9个base_anchor的基础上生成hh*ww*9个anchor 对应到原坐标
    
    n_anchor = anchor.shape[0] // (hh * ww) #hh*ww*9/hh*ww = 9
    h = F.relu(self.conv1(x)) #512个3*3卷积 (512,H/16,W/16)

    rpn_locs = self.loc(h) # n_anchor(9)*4个1*1卷积，回归坐标偏移量 (9*4, hh, ww)

    #转换为(n, hh, ww, 9*4)后变为(n, hh*ww*9, 4)
    rpn_locs = rpn_locs.permute(0, 2, 3, 1).contiguous().view(n, -1, 4)
    rpn_scores = self.score(h) # n_anchor(9)*2个1*1卷积 回归类别 (9*2, hh, ww)
    rpn_scores = rpn_scores.permute(0, 2, 3, 1).contiguous() # 转换为(n, hh, ww, 9*2)
    rpn_softmax_scores = F.softmax(rpn_scores.view(n, hh, ww, n_anchor, 2), dim=4) #计算softmax
    rpn_fg_scores = rpn_softmax_scores[:,:,:,:,1].contiguous() #得到前景的分类概率
    rpn_fg_scores = rpn_fg_scores.view(n, -1) # 得到所有anchor的前景分类概率
    rpn_scores = rpn_scores.view(n, -1, 2) #得到每一张feature map上所有anchor的网络输出值

    rois = list()
    roi_indices = list()
    for i in range(n): # n为batch_size数

    # 调用ProposalCreator函数，rpn_locs维度(hh*ww*9,4)，rpn_fg_scores维度为(hh*ww*9),
    # anchor的维度为(hh*ww*9, 4), img_size的维度为(3,H,W),W和H是经过数据处理后的
    # 计算(H/16)*(W/16)*9 大概20000个anchor属于前景的概率，取前12000个并经过NMS得到2000个近似目标框

      roi = self.proposal_layer(
          rpn_locs[i].cpu().data.numpy(), # 位置信息
          rpn_fg_scores[i].cpu().data.numpy(), # 前景概率
          anchor,img_size,
          scale=scale)
      
      batch_index = i * np.ones((len(roi),),dtype=np.int32)
      rois.append(roi)
      roi_indices.append(batch_index)

    rois = np.concatenate(rois, axis=0) # rois为所有batch_size的roi
    roi_indices = np.concatenate(roi_indices, axis=0) # 按行拼接
    # rpn_locs的维度(hh*ww*9,4), rpn_scores维度为(hh*ww*9,2)
    # rois的维度(2000,4), roi_indices用不到, anchor的维度为(hh*ww*9, 4)
    return rpn_locs, rpn_scores, rois, roi_indices,anchor

def _enumerate_shifted_anchor(anchor_base, feat_stride, height, width):
  # 在整个特征图生成所有的anchor大小，并对应回原图大小
  import numpy as xp
  shift_y = xp.arange(0, height * feat_stride,feat_stride) #纵向偏移量
  shift_x = xp.arange(0, width * feat_stride,feat_stride) #横向偏移量
  #shift_x = [[0,16,32, ...],[0, 16, 32, ...],[0, 16, 32, ...]...],
  #shift_y = [[0,0,0, ...],[16, 16, 16, ...],[32, 32, 32, ...]...],
  #就是形成了一个纵横偏移量的矩阵，也就是特征图每一个点都能通过这个矩阵找到映射在原图中的具体位置
  shift_x, shift_y = xp.meshgrid(shift_x, shift_y)
  # 产生偏移坐标对，一个朝x方向 一个朝y方向
  shift = xp.stack((shift_y.ravel(),shift_x.ravel(),
                    shift_y.ravel(),shift_x.ravel()), axis=1)
  # 用基础的9个anchor的坐标分别和偏移量相加，最后得出了所有anchor的坐标，四列可以看成是左上角的坐标和右下角的坐标加偏移量的同步执行，
  # 飞速的从上往下捋一遍，所有的anchor就都出来了，一共K个特征点 每一个有A个基本的anchor 最后reshape((K*A),A)的形式，也就得到了最后的所有的anchor
  # 左下角和右上角坐标
  A = anchor_base.shape[0] # A = 9
  K = shift.shape[0] # 读取特征图中元素的总个数
  anchor = anchor_base.reshape((1, A, 4)) + \
        shift.reshape((1, K, 4)).transpose((1,0,2)) #(k,1,4)
  # np的广播机制
  anchor = anchor.reshape((K * A, 4)).astype(np.float32)
  return anchor

def normal_init(m,mean,stddev,truncated=False): #初始化数值的方法

  if truncated:
    m.weight.data.normal_().fmod_(2).mul_(stddev).add_(mean)
  else:
    m.weight.data.normal_(mean,stddev)
    m.bias.data.zero_()

# Faster R-CNN

In [ ]:
def decom_vgg16():
  # the 30th layer of features is relu of con5_3
  if opt.caffe_pretrain: # 是否使用caffe预训练
    model = vgg16(pretrain=False)
    if not opt.load_path:
      model.load_state_dict(t.load(opt.caffe_pretrain_path))

  else:
    model = vgg(not opt.load_path)

  features = list(model.features)[:30] #加载预训练模型vgg16的con5_3之前的部分
  classifier = model.classifier

  classifier = list(classifier)
  del classifier[6] #删除最后的1000类分类层

  if not opt.use_drop: #删除最后两个dropout层
    del classifier[5] 
    del classifier[2] 
  classifier = nn.Sequential(*classifier)

  for layer in features[:10]: #冻结vgg16前两个stage 不进行反向传播
    for p in layer.parameters():
      p.requiers_grad = False

  return nn.Sequential(*features), classifier # 拆分为特征提取网络和分类网络


class FasterRCNNVGG16(FasterRCNN):
  #分别对特征VGG16的特征提取部分 分类部分 RPN网络 VGG16RoIHead网络进行实例化
  feat_stride = 16

  def __init__(self,
               n_fg_class=20,
               ratios=[0.5,1,2],
               anchor_scales=[8,16,32]
               ): # 总类别数是20类 三种尺度三种比例的anchor == 9
    
    extractor,classifier = decom_vgg16() # conv5_3及之前的部分，分类器

    rpn = RegionProposalNetwork(
        512,512,
        ratios = ratios,
        anchor_scales = anchor_scales,
        feat_stride = feat_stride,
    ) #返回rpn_locs, rpn_scores, rois, roi_indices, anchor

    head = VGG16RoIHead(
        n_class = n_fg_class + 1,
        roi_size = 7,
        spatial_scale = (1./self.feat_stride),
        classifier=classifier
    ) # 下面会分析VGG16RoIHead 

    super(FasterRCNNVGG16,self).__init__(
        extractor,
        rpn,
        head,
    ) #相当于给fasterRCNN传入参数 extractor rpn head


class VGG16RoIHead(nn.Module):
  def __init__(self, n_class, roi_size, spatial_scale,
               classifier):
    # n_class includes the background
    super(VGG16RoIHead, self).__init__()

    self.classifier = classifier # vgg16中的classifier
    self.cls_loc = nn.Linear(4096, n_class*4)
    self.score = nn.Linear(4096, n_class)

    normal_init(self.cls_loc, 0.001)
    normal_init(self.score, 0.001) #全连接层权重初始化

    self.n_class = n_class #加上背景21类
    self.roi_size = roi_size # 7
    self.spatial_scale = spatial_scale # 1/16
    #将不同大小的roi变成大小一致，得到pooling后的特征，大小为[300,512,7,7] 利用Cupy实现在线编译的
    #使用的是from torchvision.ops import RoIPool
    self.roi = RoIPool((self.roi_size, self.roi_size),self.spatial_scale)

  def forward(self, x, rois, roi_indices):

    roi_indices = at.totensor(roi_indices).float() # ndarray -> tensor
    rois = at.totensor(rois).float()
    indices_and_rois = t.cat([roi_indices[:,None], rois],dim=1) 

    xy_indices_and_rois = indices_and_rois[:,[0,2,1,4,3]]
    indices_and_rois = xy_indices_and_rois.contiguous() #把tensor变成在内存中连续分布的形式

    pool = self.roi(x, indices_and_rois) # RoIPooling
    pool = pool.view(pool.size(0), -1)
    fc7 = self.classifier(pool) # decom_vgg16()得到的classifier 得到4096
    roi_clas_locs = self.cls_loc(fc7) # 4096->84 84=21*4
    roi_scores = self.score(fc7) # 4096->21

    return roi_cls_locs, roi_scores


  def normal_init(m, mean, stddev, truncated=False):#只有正数的初始化和随机初始化
    if truncated:
      m.weight.data.normal_().fmod_(2).mul_(stddev).add_(mean)

    else:
      m.weight.data.normal_(mean,stddev)
      m.bias.data.zero_()

